In [1]:
import pandas as pd
import datetime
import re

In [2]:
data_oldcust = pd.read_csv("D:\\DA_Camp\\old_cust.csv")
cust_info = pd.read_csv("D:\\DA_Camp\\customer_information.csv")

#  I. Tạo bảng not_first_time để đếm số đơn hàng từng ngày từ khách hàng cũ

In [ ]:
'''
1. Từ bảng data_oldcust, tạo ra bảng mua hàng lần đầu (gọi là first_time) của các Id như đã làm ở file check.ipynb
2. bảng not_first_time = những gì còn lại sau khi loại những dòng của bảng first_time ra khỏi bảng data_oldcust 

** Bảng data_oldcust là bảng bao gồm tất cả những lần mua hàng của mọi khách hàng kèm theo ngày mua 
'''

In [27]:
data_oldcust

,visitor_id,noOrders,create_date
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
4,++P59XK9q+K1acy8MeRVzfrfOz0=,2,2019-11-13
...,...,...,...
88979,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
88980,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
88981,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14
88982,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-16


In [19]:
# Lấy bảng first_time ra và giữ nguyên index
first_time = data_oldcust.drop_duplicates(subset= "visitor_id", keep="first")
first_time

,visitor_id,noOrders,create_date
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
7,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04
...,...,...,...
88973,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08
88979,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
88980,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
88981,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14


In [20]:
# Từ bảng data_old_cust, loại những index trong bảng first_time 
# Sau đó sort lại cho đỡ lộn xộn
not_first_time = data_oldcust.drop(labels= first_time.index)
not_first_time.sort_values(by=['visitor_id', 'create_date'], ascending= [1,1], ignore_index= True,inplace= True)
not_first_time

,visitor_id,noOrders,create_date
0,++P59XK9q+K1acy8MeRVzfrfOz0=,2,2019-11-13
1,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-18
2,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-12-07
3,++Rc3Z/cGqApeiipO78fSuUXrdo=,2,2019-12-25
4,++Rc3Z/cGqApeiipO78fSuUXrdo=,2,2019-12-27
...,...,...,...
49540,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-22
49541,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-23
49542,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-30
49543,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-12-05


In [30]:
# Lưu file
not_first_time.to_csv("D:\\DA_Camp\\not_first_time.csv",index = False)

# II. Tính tỷ lệ chuyển đổi ngay lần đầu khách hàng biết đến công ty

In [ ]:
'''
Conversion_rate hàng ngày = lượng người mua hàng / lượng người ghé qua, hàng ngày
Tuy nhiên chúng ta chỉ có dữ liệu về lần đầu 1 khách hàng ghé qua chứ không biết những ngày sau đó họ có ghé lại hay không
Vì vậy, ta chỉ có thể tính Conversion_rate_when_first_visit hàng ngày = lượng người ghé qua lần đầu và mua hàng / lượng nguồi ghé qua lần đầu,hàng ngày
'''

'''
Để tính lượng người ghé qua lần đầu và mua hàng hàng ngày:
    1. Từ bảng customer_information, lấy ra những Id có first_buy và first_visit bằng nhau
    2. Đếm lượng Id từng ngày và chia theo visitor_source để khi visualize có thêm góc nhìn
Lúc này ta có bảng first_visitbuy
'''

'''
Để tính lượng người ghé qua lần đầu hàng ngày: Từ bảng visitor_information, đếm lượng Id từng ngày và cũng chia theo visitor_source
Lúc này ta có bảng first_visit
'''

'''
bảng conversion_rate = bảng First_VisitBuy merge với bảng First_Visit
'''

In [6]:
# Lấy ra những Id có first_buy = first_time
first_visitbuy = cust_info.loc[cust_info['first_buy'] == cust_info['first_visit']]
first_visitbuy


,visitor_id,noOrders,first_buy,first_visit,gender,age,visitor_source,type
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty,Khách lẻ
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty,Khách lẻ
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty,Khách lẻ
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,Không rõ,Website công ty,Khách lẻ
6,++eXAqAXiEeaVqRLmrOElOIgwi4=,1,2019-11-11,2019-11-11,Không rõ,Không rõ,Website công ty,Khách lẻ
...,...,...,...,...,...,...,...,...
39433,zz+3mUNMhQ10EBbMg/8WItiohU8=,4,2019-11-13,2019-11-13,Nữ,17.0,Website công ty,Khách lẻ
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,Không rõ,Website công ty,Khách lẻ
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,Không rõ,Website công ty,Khách lẻ
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,Không rõ,Website công ty,Khách lẻ


In [7]:
# Đếm lượng Id từng ngày và chia theo visitor_source
first_visitbuy = first_visitbuy.groupby(['first_buy', 'visitor_source']).count().reset_index()
first_visitbuy

,first_buy,visitor_source,visitor_id,noOrders,first_visit,gender,age,type
0,2019-11-01,Cừa hàng,50,50,50,50,50,50
1,2019-11-01,Sàn thương mại điện tử,16,16,16,16,16,16
2,2019-11-01,Website công ty,346,346,346,346,346,346
3,2019-11-02,Cừa hàng,52,52,52,52,52,52
4,2019-11-02,Sàn thương mại điện tử,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...
291,2020-02-07,Sàn thương mại điện tử,3,3,3,3,3,3
292,2020-02-07,Website công ty,39,39,39,39,39,39
293,2020-02-08,Cừa hàng,18,18,18,18,18,18
294,2020-02-08,Sàn thương mại điện tử,9,9,9,9,9,9


In [8]:
# Bỏ những cột không cần thiết và sửa tiêu đề cột cho đúng
first_visitbuy = first_visitbuy.drop(labels= first_visitbuy.columns[3:], axis=1).rename({'visitor_id' : 'First_VisitBuy', 'first_buy' : 'Date'},axis=1)
first_visitbuy

,Date,visitor_source,First_VisitBuy
0,2019-11-01,Cừa hàng,50
1,2019-11-01,Sàn thương mại điện tử,16
2,2019-11-01,Website công ty,346
3,2019-11-02,Cừa hàng,52
4,2019-11-02,Sàn thương mại điện tử,17
...,...,...,...
291,2020-02-07,Sàn thương mại điện tử,3
292,2020-02-07,Website công ty,39
293,2020-02-08,Cừa hàng,18
294,2020-02-08,Sàn thương mại điện tử,9


In [10]:
# load file visitor_information để tạo bảng first_visit
visitor = pd.read_csv("D:\\DA_Camp\\visitor_information.csv")

In [11]:
# Đếm lượng Id từng ngày và chia theo visitor_source
first_visit = visitor.groupby(['create_date', 'visitor_source']).count().reset_index()
# Xóa những cột không cần thiết
first_visit = first_visit.drop(labels= first_visit.columns[3:], axis=1)
first_visit

,create_date,visitor_source,visitor_id
0,2019-11-01,Cừa hàng,170
1,2019-11-01,Sàn thương mại điện tử,56
2,2019-11-01,Website công ty,1972
3,2019-11-02,Cừa hàng,197
4,2019-11-02,Sàn thương mại điện tử,62
...,...,...,...
295,2020-02-07,Sàn thương mại điện tử,71
296,2020-02-07,Website công ty,180
297,2020-02-08,Cừa hàng,78
298,2020-02-08,Sàn thương mại điện tử,36


In [12]:
# Đổi tiêu đề cột cho đúng
first_visit = first_visit.rename({'visitor_id' : 'First_Visit', 'create_date' : 'Date'},axis=1)
first_visit

,Date,visitor_source,First_Visit
0,2019-11-01,Cừa hàng,170
1,2019-11-01,Sàn thương mại điện tử,56
2,2019-11-01,Website công ty,1972
3,2019-11-02,Cừa hàng,197
4,2019-11-02,Sàn thương mại điện tử,62
...,...,...,...
295,2020-02-07,Sàn thương mại điện tử,71
296,2020-02-07,Website công ty,180
297,2020-02-08,Cừa hàng,78
298,2020-02-08,Sàn thương mại điện tử,36


In [13]:
# merge bảng first_visitbuy với bảng first_visit để tạo thành bảng conversion_rate 
conversion_rate = pd.merge(first_visitbuy, first_visit, how= "outer", on= ['Date', 'visitor_source'])
conversion_rate

,Date,visitor_source,First_VisitBuy,First_Visit
0,2019-11-01,Cừa hàng,50.0,170
1,2019-11-01,Sàn thương mại điện tử,16.0,56
2,2019-11-01,Website công ty,346.0,1972
3,2019-11-02,Cừa hàng,52.0,197
4,2019-11-02,Sàn thương mại điện tử,17.0,62
...,...,...,...,...
295,2020-02-08,Website công ty,35.0,171
296,2019-12-14,Sàn thương mại điện tử,NaN,38
297,2020-01-26,Sàn thương mại điện tử,NaN,4
298,2020-01-27,Sàn thương mại điện tử,NaN,3


In [14]:
# Có những ngày có người ghé thăm lần đầu nhưng không ai vừa ghé thăm lần đầu và vừa mua. Nên cột 'First_VisitBuy' có những giá trị NaN. Thay những chỗ đó bằng 0
conversion_rate['First_VisitBuy'] = conversion_rate['First_VisitBuy'].fillna(0)
conversion_rate

,Date,visitor_source,First_VisitBuy,First_Visit
0,2019-11-01,Cừa hàng,50.0,170
1,2019-11-01,Sàn thương mại điện tử,16.0,56
2,2019-11-01,Website công ty,346.0,1972
3,2019-11-02,Cừa hàng,52.0,197
4,2019-11-02,Sàn thương mại điện tử,17.0,62
...,...,...,...,...
295,2020-02-08,Website công ty,35.0,171
296,2019-12-14,Sàn thương mại điện tử,0.0,38
297,2020-01-26,Sàn thương mại điện tử,0.0,4
298,2020-01-27,Sàn thương mại điện tử,0.0,3


In [15]:
# Lưu file
conversion_rate.to_csv("D:\\DA_Camp\\conversion_rate.csv",index = False)

# III. Tính khoảng thời gian trung bình 1 người mua hàng lần nữa

In [ ]:
'''
1. Tính số lần mua
    - Từ bảng data_oldcust, đếm số lần mua của từng Id
    - Vì em đang muốn tính khoảng thời gian trung bình để 1 người mua hàng lần nữa nên em bỏ đi nhũng Id có số lần mua bằng 1
2. Tính khoảng thời gian giữa lần cuối và lần đầu khách mua hàng
    - Từ bảng not_first_time, bỏ đi duplicate của cột visitor_id và giữ lại dòng cuối cùng trong mỗi nhóm duplicate ( vì bảng này đã được sort theo cột create_date, ascending)
      như vậy là có được mốc last_time của 1 Id
    - Lấy mốc last_time đó trừ đi mốc first_time đã có ở bảng customer_information
'''

In [16]:
df = data_oldcust.groupby(['visitor_id']).count().reset_index()
df = df.drop(labels= df.columns[2:], axis = 1).rename({'noOrders' : 'NoBuy'}, axis = 1,)
df

,visitor_id,NoBuy
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,1
1,++HN809MR7t8V9QmdOEA36NubfI=,1
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1
3,++P59XK9q+K1acy8MeRVzfrfOz0=,4
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,1
...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,6
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,2


In [17]:
df = df.drop(labels= df.loc[df['NoBuy'] == 1].index)
df = df.sort_values(by = 'visitor_id', ignore_index= True)
df

,visitor_id,NoBuy
0,++P59XK9q+K1acy8MeRVzfrfOz0=,4
1,++Rc3Z/cGqApeiipO78fSuUXrdo=,6
2,++oEHFOpZPhAsTkYw1CVb/+AYrU=,5
3,++sf+nIMstu6uI9e53II6GyjGQU=,2
4,++zohmaabt4uh3LNxdHGxOcdoT8=,2
...,...,...
15513,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,6
15514,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,3
15515,zz+3mUNMhQ10EBbMg/8WItiohU8=,4
15516,zz4YKQwMrdb6za9UWGnbxWHcLZY=,6


In [21]:
last_time = not_first_time.drop_duplicates(subset='visitor_id', keep='last')
last_time


,visitor_id,noOrders,create_date
2,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-12-07
7,++Rc3Z/cGqApeiipO78fSuUXrdo=,1,2020-02-02
11,++oEHFOpZPhAsTkYw1CVb/+AYrU=,1,2019-12-01
12,++sf+nIMstu6uI9e53II6GyjGQU=,1,2019-12-05
13,++zohmaabt4uh3LNxdHGxOcdoT8=,1,2020-01-10
...,...,...,...
49533,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,1,2020-01-24
49535,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,1,2020-01-09
49538,zz+3mUNMhQ10EBbMg/8WItiohU8=,1,2019-12-18
49543,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-12-05


In [22]:
last_time = last_time.rename({'create_date': 'last_buy'},axis=1)
last_time = last_time.sort_values(by = 'visitor_id', ignore_index= True)
last_time

,visitor_id,noOrders,last_buy
0,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-12-07
1,++Rc3Z/cGqApeiipO78fSuUXrdo=,1,2020-02-02
2,++oEHFOpZPhAsTkYw1CVb/+AYrU=,1,2019-12-01
3,++sf+nIMstu6uI9e53II6GyjGQU=,1,2019-12-05
4,++zohmaabt4uh3LNxdHGxOcdoT8=,1,2020-01-10
...,...,...,...
15513,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,1,2020-01-24
15514,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,1,2020-01-09
15515,zz+3mUNMhQ10EBbMg/8WItiohU8=,1,2019-12-18
15516,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-12-05


In [23]:
last_time = pd.merge(last_time, cust_info, how = "left", on= 'visitor_id')
last_time


,visitor_id,noOrders_x,last_buy,noOrders_y,first_buy,first_visit,gender,age,visitor_source,type
0,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-12-07,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty,Khách lẻ
1,++Rc3Z/cGqApeiipO78fSuUXrdo=,1,2020-02-02,1,2019-12-24,2019-12-23,Nữ,28.0,Cừa hàng,Khách lẻ
2,++oEHFOpZPhAsTkYw1CVb/+AYrU=,1,2019-12-01,2,2019-11-23,2019-11-23,Nữ,14.0,Website công ty,Khách lẻ
3,++sf+nIMstu6uI9e53II6GyjGQU=,1,2019-12-05,1,2019-12-02,2019-11-29,LGBT,18.0,Website công ty,Khách lẻ
4,++zohmaabt4uh3LNxdHGxOcdoT8=,1,2020-01-10,1,2019-11-17,2019-11-17,Nam,30.0,Website công ty,Khách lẻ
...,...,...,...,...,...,...,...,...,...,...
15513,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,1,2020-01-24,1,2019-11-20,2019-11-03,Nam,25.0,Website công ty,Khách lẻ
15514,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,1,2020-01-09,1,2020-01-02,2020-01-02,Không rõ,Không rõ,Website công ty,Khách lẻ
15515,zz+3mUNMhQ10EBbMg/8WItiohU8=,1,2019-12-18,4,2019-11-13,2019-11-13,Nữ,17.0,Website công ty,Khách lẻ
15516,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-12-05,1,2019-11-08,2019-11-01,Nữ,Không rõ,Website công ty,Khách lẻ


In [24]:
list = []
for i in last_time.index:
    first = datetime.datetime.strptime(last_time.loc[i]['first_buy'],"%Y-%m-%d").date()
    last = datetime.datetime.strptime(last_time.loc[i]['last_buy'],"%Y-%m-%d").date()
    tdelta = last - first
    list.append(int(re.findall('[0-9]+' ,str(tdelta))[0]))


In [25]:
df['tdelta'] = list
df

,visitor_id,NoBuy,tdelta
0,++P59XK9q+K1acy8MeRVzfrfOz0=,4,25
1,++Rc3Z/cGqApeiipO78fSuUXrdo=,6,40
2,++oEHFOpZPhAsTkYw1CVb/+AYrU=,5,8
3,++sf+nIMstu6uI9e53II6GyjGQU=,2,3
4,++zohmaabt4uh3LNxdHGxOcdoT8=,2,54
...,...,...,...
15513,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,6,65
15514,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,3,7
15515,zz+3mUNMhQ10EBbMg/8WItiohU8=,4,35
15516,zz4YKQwMrdb6za9UWGnbxWHcLZY=,6,27


In [27]:
df.to_csv('D:\\DA_Camp\\avg_days_for_repurchase.csv',index= False)

# IV. Tính tỷ lệ giữ chân khách hàng trong 1 period

In [ ]:
# Tạo biến period để nếu có muốn chuyển chu kỳ thành 50 ngày hay 14 ngày hay bất cứ số nào cũng chỉ cần chuyển ở biến period này
period = 30

## 1.Ghép period vào bảng data_oldcust
(Bảng data_oldcust ghi lại tất cả các lần mua của khách hàng và ngày mua)

In [119]:
df = data_oldcust.sort_values(by= 'create_date', ignore_index= True)
df

,visitor_id,noOrders,create_date
0,262uP5ixn2svd0vnlmZ8Asg0ckk=,1,2019-11-01
1,eKTU6a85IMYviAUneppyKDPwWJg=,1,2019-11-01
2,cLnneqkMyW0owR36QeSAyEFG9z4=,1,2019-11-01
3,MS36D4J+qmMhmrBWsr4yhVh3Bxk=,5,2019-11-01
4,KlTbKOBkNrppmmS9zO0fwduFKlw=,1,2019-11-01
...,...,...,...
88979,ahcCUOV0Y4Y9OsMBGYKjVH3hNzY=,1,2020-02-08
88980,U0x55gTkTC5cHeGCdSXm+xbFJkM=,1,2020-02-08
88981,qYu5dMtumvsNY5e0l+6b0acyEcI=,2,2020-02-08
88982,A+Jg0cZ57UP5CZy7Kc7Y0h/1/rE=,2,2020-02-08


In [2]:
x = datetime.date(2019,11,1)
y = datetime.date(2020,2,8)
y-x

datetime.timedelta(days=99)

In [120]:
value_of_column = []

interval_of_a_period = datetime.timedelta(days = period)
period_number = 1
first_date_in_period = datetime.datetime.strptime(df.loc[0]['create_date'], "%Y-%m-%d").date()

for i in df.index:
    if datetime.datetime.strptime(df.loc[i]['create_date'], "%Y-%m-%d").date() - first_date_in_period + datetime.timedelta(days = 1) > interval_of_a_period:
        first_date_in_period = datetime.datetime.strptime(df.loc[i]['create_date'], "%Y-%m-%d").date()
        period_number += 1
    if datetime.datetime.strptime(df.loc[len(df) - 1]['create_date'], "%Y-%m-%d").date() - first_date_in_period + datetime.timedelta(days = 1) < interval_of_a_period:
        break
    value_of_column.append('period' +'_'+ str(period_number))

for i in range(0, (len(df) - len(list)), 1):
    value_of_column.append("Not enough")

In [122]:
df['period'] = value_of_column
df

,visitor_id,noOrders,create_date,period
0,262uP5ixn2svd0vnlmZ8Asg0ckk=,1,2019-11-01,period_1
1,eKTU6a85IMYviAUneppyKDPwWJg=,1,2019-11-01,period_1
2,cLnneqkMyW0owR36QeSAyEFG9z4=,1,2019-11-01,period_1
3,MS36D4J+qmMhmrBWsr4yhVh3Bxk=,5,2019-11-01,period_1
4,KlTbKOBkNrppmmS9zO0fwduFKlw=,1,2019-11-01,period_1
...,...,...,...,...
88979,ahcCUOV0Y4Y9OsMBGYKjVH3hNzY=,1,2020-02-08,Not enough
88980,U0x55gTkTC5cHeGCdSXm+xbFJkM=,1,2020-02-08,Not enough
88981,qYu5dMtumvsNY5e0l+6b0acyEcI=,2,2020-02-08,Not enough
88982,A+Jg0cZ57UP5CZy7Kc7Y0h/1/rE=,2,2020-02-08,Not enough


## 2. Tính số khách hàng ở đầu mỗi kỳ

In [123]:
df = df.drop_duplicates(subset= ['visitor_id', 'period'], keep= 'first')
df

,visitor_id,noOrders,create_date,period
0,262uP5ixn2svd0vnlmZ8Asg0ckk=,1,2019-11-01,period_1
1,eKTU6a85IMYviAUneppyKDPwWJg=,1,2019-11-01,period_1
2,cLnneqkMyW0owR36QeSAyEFG9z4=,1,2019-11-01,period_1
3,MS36D4J+qmMhmrBWsr4yhVh3Bxk=,5,2019-11-01,period_1
4,KlTbKOBkNrppmmS9zO0fwduFKlw=,1,2019-11-01,period_1
...,...,...,...,...
88975,hjJoHjHM9h0ng4PWswJsWK16UV4=,1,2020-02-08,Not enough
88976,DVbNrS2cdxYo50Xa3h/ptNWT+JY=,4,2020-02-08,Not enough
88977,w3q2i2BzRarTj6N8MGZKDq1nCqU=,1,2020-02-08,Not enough
88981,qYu5dMtumvsNY5e0l+6b0acyEcI=,2,2020-02-08,Not enough


In [124]:
df = df.drop(labels = df[df['period'] == "Not enough"].index)
df

,visitor_id,noOrders,create_date,period
0,262uP5ixn2svd0vnlmZ8Asg0ckk=,1,2019-11-01,period_1
1,eKTU6a85IMYviAUneppyKDPwWJg=,1,2019-11-01,period_1
2,cLnneqkMyW0owR36QeSAyEFG9z4=,1,2019-11-01,period_1
3,MS36D4J+qmMhmrBWsr4yhVh3Bxk=,5,2019-11-01,period_1
4,KlTbKOBkNrppmmS9zO0fwduFKlw=,1,2019-11-01,period_1
...,...,...,...,...
82751,F26riPmG4LPCfLmR03Vo0ingaNc=,1,2020-01-29,period_3
82752,uwnoEi8fVnl+H27lbWudJtai0kw=,1,2020-01-29,period_3
82756,EGp2AdEeKf289rKQ9zTl6fNjn0c=,1,2020-01-29,period_3
82757,ZabcC+9K8QAqvMXv6mjM77hVBCk=,5,2020-01-29,period_3


In [125]:
cust_retention_rate = df.groupby('period').count().reset_index()
cust_retention_rate =  cust_retention_rate.drop(columns= cust_retention_rate.columns[2:])
cust_retention_rate

,period,visitor_id
0,period_1,20587
1,period_2,14494
2,period_3,9010


In [126]:
list = [0]
for i in cust_retention_rate.index:
    if i > 0:
        list.append(cust_retention_rate.loc[i - 1]['visitor_id'])
cust_retention_rate['begin_of_period'] = list
cust_retention_rate

,period,visitor_id,begin_of_period
0,period_1,20587,0
1,period_2,14494,20587
2,period_3,9010,14494


In [127]:
cust_retention_rate = cust_retention_rate.drop(columns= 'visitor_id')
cust_retention_rate

,period,begin_of_period
0,period_1,0
1,period_2,20587
2,period_3,14494


## 3. Tính số khách mất đi trong mỗi kỳ

In [128]:
list = [0]
for i in range(1, len(df['period'].unique()) + 1):
    count = 0
    if i > 1:
        a = df.loc[df['period'] == 'period' +'_'+ str(i - 1)]['visitor_id'].unique()
        b = df.loc[df['period'] == 'period' +'_'+ str(i)]['visitor_id'].unique()
        for j in range(len(a)):
            if a[j] not in b:
                count += 1
        list.append(count)
cust_retention_rate['lost'] = list
cust_retention_rate

,period,begin_of_period,lost
0,period_1,0,0
1,period_2,20587,17441
2,period_3,14494,11551


## 4. Xuất file

In [129]:
cust_retention_rate.to_csv("D:\\DA_Camp\\cust_retention_rate_{period}.csv".format(period = period),index = False)

# **Phân loại khách buôn và khách lẻ để có thêm góc nhìn

In [146]:
a = data_oldcust.sort_values(by = ['visitor_id', 'noOrders'], ascending= [0,0]).drop_duplicates(subset = 'visitor_id', keep = 'first')
a = a.loc[a['noOrders'] > 100]
a


,visitor_id,noOrders,create_date
71391,nFKxoKPev4LVn/GCXfrtrqMoie8=,189,2020-01-30
66613,jnPbGy6KfUEod9Pr/Fqh3awXxoQ=,276,2019-11-03
55499,bpkDQAjlBFfXRZnBgCzj0xvtEmQ=,489,2019-11-03
43936,TKfm2Xfj5UcOwah0O9JNedIYKUQ=,169,2019-11-02
26296,Gm2OVIM5L14awx+VStQWIvSEOhc=,253,2019-11-03
4649,1BV9gNJd11mnvjm0pzXAIqDlUDw=,340,2019-11-03
3228,0JU+j9QCFOCX62uwU9R6OmWxMT8=,129,2020-01-23


In [147]:
list = []
for i in range(len(a)):
    list.append("Khách buôn")
a['type'] = list


In [148]:
a = a.drop(columns= ['noOrders', 'create_date'])
a


,visitor_id,type
71391,nFKxoKPev4LVn/GCXfrtrqMoie8=,Khách buôn
66613,jnPbGy6KfUEod9Pr/Fqh3awXxoQ=,Khách buôn
55499,bpkDQAjlBFfXRZnBgCzj0xvtEmQ=,Khách buôn
43936,TKfm2Xfj5UcOwah0O9JNedIYKUQ=,Khách buôn
26296,Gm2OVIM5L14awx+VStQWIvSEOhc=,Khách buôn
4649,1BV9gNJd11mnvjm0pzXAIqDlUDw=,Khách buôn
3228,0JU+j9QCFOCX62uwU9R6OmWxMT8=,Khách buôn


In [149]:
cust_info =  pd.merge(cust_info, a, how= 'left', on = 'visitor_id')
cust_info


,visitor_id,noOrders,first_buy,first_visit,gender,age,visitor_source,type
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12,2020-01-10,LGBT,21.0,Website công ty,NaN
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty,NaN
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty,NaN
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty,NaN
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,Không rõ,Website công ty,NaN
...,...,...,...,...,...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08,2019-11-01,Nữ,Không rõ,Website công ty,NaN
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,Không rõ,Website công ty,NaN
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,Không rõ,Website công ty,NaN
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,Không rõ,Website công ty,NaN


In [150]:
cust_info['type'] = cust_info['type'].fillna('Khách lẻ')
cust_info

,visitor_id,noOrders,first_buy,first_visit,gender,age,visitor_source,type
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12,2020-01-10,LGBT,21.0,Website công ty,Khách lẻ
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty,Khách lẻ
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty,Khách lẻ
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty,Khách lẻ
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,Không rõ,Website công ty,Khách lẻ
...,...,...,...,...,...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08,2019-11-01,Nữ,Không rõ,Website công ty,Khách lẻ
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,Không rõ,Website công ty,Khách lẻ
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,Không rõ,Website công ty,Khách lẻ
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,Không rõ,Website công ty,Khách lẻ


In [151]:
cust_info.to_csv("D:\\DA_Camp\\customer_information.csv", index= False)